# Quick introduction

erddapy is a pure python package and can be installed with conda

```shell
conda install --channel conda-forge erddapy
```

or pip

```shell
pip install erddapy
```


First we need to instantiate the ERDDAP URL constructor for a server. In this
example we will use
[https://upwell.pfeg.noaa.gov/erddap](https://upwell.pfeg.noaa.gov/erddap/index.html).


In [ ]:
from erddapy import ERDDAP


e = ERDDAP(
    server="UAF",  # NOAA UAF (Unified Access Framework)
    protocol="tabledap",
    response="csv",
)

Now we can populate the object a dataset id, variables of interest, and its
constraints (last week gliders). Use the method `to_pandas` to download the
csv(p) response, a comma separated values with units and explore the Dataframe.


In [ ]:
e.dataset_id = "scrippsGliders"

e.variables = [
    "depth",
    "latitude",
    "longitude",
    "salinity",
    "temperature",
    "time",
]

e.constraints = {
    "time>=": "now-7days",
}


df = e.to_pandas(
    index_col="time (UTC)",
    parse_dates=True,
).dropna()

df.head()

We can constraint the in time and space with relative constraints like in the
example below. For more ways to access the data please check the "Longer
introduction."


In [ ]:
constraints = {
    "time>=": "now-14days",
    "time<=": "now-7days",
    "latitude>=": "min(latitude)+5",
    "latitude<=": "max(latitude)-5",
    "longitude>=": "min(longitude)+5",
    "longitude<=": "min(longitude)+10",
    "depth>=": "min(depth)+5",
    "depth<=": "max(depth)-40",
}


url = e.get_download_url(
    response="html",
    constraints=constraints,
)

print(url)

The same request as above but with non-relative constraints. Note that these
values will change if we run this at a later time.


In [ ]:
constraints = {
    "time>=": "2021-09-11T23:26:55Z",
    "time<=": "2021-09-18T23:26:55Z",
    "latitude>=": -7.52,
    "latitude<=": 46.67,
    "longitude>=": -121.48,
    "longitude<=": -116.48,
    "depth>=": -3.82,
    "depth<=": 1001.59,
}

url = e.get_download_url(
    response="html",
    constraints=constraints,
)

print(url)

We can search all datasets with a set of constraints by setting `dataset_id` to `"allDatasets"`. Note that these variables are different than the ones available at the individual dataset level. For a reference of the possible variables to query all datasets see the `<server-url>/erddap/<protocol>/allDatasets.html` page, like [this one](https://upwell.pfeg.noaa.gov/erddap/tabledap/allDatasets.html) for the UAF ERDDAP server.

In [ ]:
e.dataset_id = "allDatasets"

e.variables = [
    "datasetID",
    "institution",
    "minTime",
    "maxTime",
    "minLatitude",
    "maxLatitude",
    "minLongitude",
    "maxLongitude",
]

e.constraints = {
    "minTime>=": "2010-01-01T00:00:00Z",
    "maxTime<=": "2011-01-01T00:00:00Z",
}


url = e.get_download_url(response="html")
print(url)

df = e.to_pandas(
    index_col="datasetID",
    parse_dates=True,
).dropna()

df.head()